<a href="https://colab.research.google.com/github/sumitasharma/MachineLearning/blob/master/mobilenet_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install np_utils
!pip install scikit-image
import keras
import skimage
from skimage import transform
from keras import backend as k
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.layers import Input
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.utils.np_utils import to_categorical
from keras.utils import np_utils

In [0]:
mobile = keras.applications.mobilenet.MobileNet()
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path+file,target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis = 0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)


In [0]:
#!mv /content/panda_pic.jfif /content/panda_pic.png
Image(filename='/content/img-india-rhino.jpg')

In [0]:
predictions = mobile.predict(prepare_image('/content/img-india-rhino.jpg'))
results = imagenet_utils.decode_predictions(predictions)
results

Transfer Learning. Fine_tuning mobilenet.

In [0]:
mobile.summary()

In [0]:
# Download 101 Objects Database
!curl -O http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz

In [0]:
!tar -C /content -zxvf /content/"101_ObjectCategories.tar.gz"

In [0]:
import numpy as np
import os
import random
import shutil

def chooseRandomFiles(dir):
  
  # print("dir is " + dir)
  # print (os.listdir(dir))
  # list all files in dir
  files = os.listdir(dir)

  # select 0.1 of the files randomly 
  # random_files = np.random.choice(files, int(len(files)*.1))
  # Select 10 random files 
  random_files = random.sample(files, 10)
  #print (random_files)
  
  return random_files
  
src_dir = "/content/101_ObjectCategories"
test_dir = "/content/ModelData/test"
train_dir = "/content/ModelData/train"

# Walk the directory tree recursively and store subdirectories
# in a list.
dir_list = []
for path, subdirs, files in os.walk(src_dir):
  for name in subdirs:
      #print(os.path.join(src_dir, name))
      dir_list.append(os.path.join(src_dir, name))
#print (dir_list)

full_path = []
# Select full source path of the files to move
for dir in dir_list:
  files = chooseRandomFiles(dir)
  for file in files:
    full_path.append(os.path.join(dir,file))
    #print(full_path)
    
# Move files from source to test

for path in full_path:
  # print(path.split(sep='/'))
  # ['', 'content', '101_ObjectCategories', 'stapler', 'image_0036.jpg']
  src_path = path
  print("src_path:" + src_path)
  temp_dest_path = path.split(sep='/')
  
  test_path = os.path.join(test_dir, temp_dest_path[-2], temp_dest_path[-1] )
  print("test_path:" + test_path)
  
  os.renames(src_path, test_path)
  

# Copy rest of the file from source to train
try:
  shutil.copytree(src_dir, train_dir)
  # Directories are the same
except shutil.Error as e:
  print('Directory not copied. Error: %s' % e)
  # Any error saying that the directory doesn't exist
except OSError as e:
  print('Directory not copied. Error: %s' % e)

In [0]:
type(mobile)

In [0]:
# mobile.summary()
# new_model = Sequential()
# for layer in mobile.layers[:-1]:
#   new_model.add(layer)
  
# for layer in new_model.layers:
#   layer.trainable = False
  

# new_model.add(Dense(102, activation = 'softmax'))

# new_model.summary()

base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(102,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)



In [0]:
model.summary()

# for layer in model.layers:
#     layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [0]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
train_datagen = ImageDataGenerator(
        #rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        '/content/ModelData/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode="categorical")

validation_generator = test_datagen.flow_from_directory(
        '/content/ModelData/test',
        target_size=(224, 224),
        batch_size=32,
        class_mode="categorical")



model.fit_generator(
        train_generator,
        steps_per_epoch=50,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=50)

In [0]:
## Get label map from train generator
label_map = (train_generator.class_indices)
label_map

## Invert the map so that you can look up by class later
inv_label_map = dict([[v,k] for k,v in label_map.items()])
inv_label_map

In [0]:
import matplotlib.pyplot as plt

#predictions = model.predict(prepare_image('/content/faces.jpg'))
predictions = model.predict(prepare_image('/content/ModelData/test/airplanes/image_0009.jpg'))

np.set_printoptions(suppress=True)
classes = predictions.argmax(axis=-1)

classes.flat[0]
print(inv_label_map[classes.flat[0]])

In [0]:
from keras.models import load_model

model.save('/content/mobilenet_fine_tuning_101_objectcategory_model.h5')